In [ ]:
import tensorflow as tf
import os

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

In [ ]:
train_dataset.class_names

In [ ]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(validation_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(AUTOTUNE)
validation_dataset = validation_dataset.prefetch(AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
                                         tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                                         tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
input_shape = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.summary()

In [ ]:
base_model.trainable = False
base_model.summary()

In [ ]:
global_average = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
inputs, labels = next(iter(train_dataset))
batched_features = base_model(inputs)
batched_features.shape

In [ ]:
averaged = global_average(batched_features)
averaged.shape

In [ ]:
classification_layer = tf.keras.layers.Dense(1)
predictions = classification_layer(averaged)
predictions.shape

In [ ]:
inputs = tf.keras.Input(input_shape)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = classification_layer(x)
full_model = tf.keras.models.Model(inputs, outputs)
full_model.summary()

In [ ]:
lr = 0.0001
full_model.compile(tf.keras.optimizers.Adam(lr),
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])


In [ ]:
full_model.evaluate(validation_dataset)

In [ ]:
initial_epochs = 10
full_model.fit(train_dataset, epochs=initial_epochs, validation_data=validation_dataset)

In [ ]:
base_model.trainable = True

In [ ]:
len(base_model.layers)

In [ ]:
for layer in base_model.layers[:100]:
  layer.trainable = False
  

In [ ]:
full_model.summary()

In [ ]:
full_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   optimizer=tf.keras.optimizers.RMSprop(lr / 10), metrics=['accuracy'])

In [ ]:
total_epochs = initial_epochs + 10
full_model.fit(train_dataset, epochs=total_epochs, initial_epoch=initial_epochs, validation_data=validation_dataset)